# Basic scraping for Guardian discussions u

In [2]:
import re 
import requests
import pickle
import json

#scrape the page and get the short url
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')
article_url = 'https://www.theguardian.com/commentisfree/2019/mar/14/panic-maths-anxiety-studying-pupils-schools'

article_url = 'https://www.theguardian.com/commentisfree/2019/mar/11/whatsapp-facebook-extreme-polarise-radicalise-mps-politicians'

article_url = 'https://www.theguardian.com/commentisfree/2019/mar/12/communities-losing-local-radio'

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)

#the short url allows us to access the discussion
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 3
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn ,'wb') as fp:
    pickle.dump(json_data,fp)

getting comments page no. 1
there are 7 pages
getting comments page no. 2
getting comments page no. 3
getting comments page no. 4
getting comments page no. 5
getting comments page no. 6
getting comments page no. 7
archiving to bx4vj.pkl


In [5]:
from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  soup.findAll('div',  {'class' : "d-comment__body"}) ]
    return raw
    
all_texts = [ html2comments(page) for page in json_data[:] ]

from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

In [6]:
[ x for x in all_comments if len(x) < 100]

['\nAs the the Tories see everything as a competition perhaps we should have a town Olympics?\n',
 '\nWait, do you just try to talk to strangers in the pub?\n',
 '\nThe Groan is passively complicit.\n',
 '\nThe U.K., the Accrington Stanley F.C. of Politics. Only about 50 years or so for the resurgence.\n',
 '\nOld Nick...leave Mercer out of this.\n',
 "\nThat's world wide order of the day mate. Control of the masses without themselves knowing it.\n",
 "\nDon't forget the balance mate. We can't have a leaning one way from all.\n",
 "\nWe're on our way to Tory paradise!\n",
 '\n"closer" not "close"...\n',
 '\nInteresting experiment the Conservative Party have foisted on us.\n',
 '\nI take it you worship London then lol\n',
 '\nThank God you never studied Humanities.\n',
 '\nHow can you tell :)\n',
 "\nIt's given, it's taken away, it can be bought ...\n",
 "\nGMG's radio division was loss-making, what else were they supposed to do?\n",
 '\nAnd got becomes gotten...\n',
 "\nI didn't even r